# GRIK TEST(GPU ver.)

## 1. Import

In [9]:
## For Dependency
import sys
import subprocess

In [10]:
## 파이썬 기본 라이브러리
import os
from datetime import datetime
import shutil

## 라이브러리가 없으면 설치를 시도하고 이미 설치되어 있으면 import만 한다.
try:
    from PIL import Image
except:
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'pip'])
    # 에러 발생한 모듈 설치
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'pillow'])
    from PIL import Image

try:
    import numpy as np
except:
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'numpy'])
    import numpy as np
    

try:
    import torch
except:
    # 에러 발생한 모듈 설치
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'torch'])
    import torch
    
try:
	import cv2
except:
    # pip 모듈 업그레이드
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'pip'])
    # 에러 발생한 모듈 설치
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'opencv-python'])
    import cv2

try:
    from skimage.metrics import structural_similarity as ssim
except:
    # 에러 발생한 모듈 설치
    subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'scikit-image'])
    from skimage.metrics import structural_similarity as ssim

try:
	from SSIM_PIL import compare_ssim
except:
	# 모듈 설치 후 import
	subprocess.check_call([sys.executable,'-m', 'pip', 'install', '--upgrade', 'SSIM-PIL'])
	from SSIM_PIL import compare_ssim

## 2. 비디오 인식

In [13]:
## 비디오 경로
video_Path = input("비디오 경로를 입력하세요: ")

## opencv 비디오 인식
video = cv2.VideoCapture(video_Path)

## 비디오가 없을 때
if not video.isOpened():
    print(f'Cannot open the video: {video_Path}')
    exit(0)

In [14]:
## 비디오 정보
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = round(video.get(cv2.CAP_PROP_FPS))

## 디렉토리 생성
save_NAME = input("저장할 이름을 입력해주세요: ")
save_DIR = f'./result/{save_NAME}'
try:
    if not os.path.exists(save_DIR):
        os.makedirs(save_DIR)
except OSError:
    print(f'Error: Cannot Find SAVE_PATH: {save_DIR}')

## 3. 비디오 처리 및 기능 구현

In [15]:
## Define Frame Counter
frameCounter = 0

## Define SSIM pre-processor
def SSIMProcessor(first, second, thisTurn):
    
    firstImg = Image.fromarray(first)
    secondImg = Image.fromarray(second)
    if(torch.cuda.is_available()):
        score = compare_ssim(firstImg, secondImg)
    else:
        score = compare_ssim(firstImg, secondImg, GPU=False)

    ## If Score is larger than 0.87, then delete Primary
    if score < 0.929:
        cv2.imwrite(f'{save_DIR}/frame{thisTurn}.jpg', second)
        print(f'#{thisTurn} Frame Saved!!')


## Video Pre-processing
while(video.isOpened()):
    ret, image = video.read()
    if (int(video.get(1)) % fps == 0):
        if (frameCounter == 0):
            Primary = image
        else :
            Secondary = image
            SSIMProcessor(Primary, Secondary, frameCounter)
            Primary = Secondary
        frameCounter += 1
    if int(video.get(1)) == length:
        break

video.release()

## Remove Video
#os.remove(video_Path)
#print("Eliminate Video!!")

#27 Frame Saved!!
#28 Frame Saved!!
#37 Frame Saved!!
#178 Frame Saved!!
#205 Frame Saved!!
#210 Frame Saved!!
#211 Frame Saved!!
#212 Frame Saved!!
#213 Frame Saved!!
#248 Frame Saved!!
#253 Frame Saved!!
#396 Frame Saved!!
#397 Frame Saved!!
#481 Frame Saved!!
#602 Frame Saved!!
#835 Frame Saved!!
#1072 Frame Saved!!
#1116 Frame Saved!!
#1117 Frame Saved!!
#1182 Frame Saved!!
#1193 Frame Saved!!
#1196 Frame Saved!!
#1197 Frame Saved!!
#1199 Frame Saved!!
#1200 Frame Saved!!
#1201 Frame Saved!!
#1202 Frame Saved!!
#1203 Frame Saved!!
#1204 Frame Saved!!
#1205 Frame Saved!!
#1207 Frame Saved!!
#1208 Frame Saved!!
#1210 Frame Saved!!
#1211 Frame Saved!!
#1212 Frame Saved!!
#1262 Frame Saved!!
#1263 Frame Saved!!
#1264 Frame Saved!!
#1266 Frame Saved!!
#1267 Frame Saved!!
#1268 Frame Saved!!
#1271 Frame Saved!!
#1307 Frame Saved!!
#1308 Frame Saved!!
#1309 Frame Saved!!
#1311 Frame Saved!!
#1312 Frame Saved!!
#1313 Frame Saved!!
#1314 Frame Saved!!
#1315 Frame Saved!!
#1316 Frame Saved!!

KeyboardInterrupt: 

## 4. PDF 변환

In [7]:
print("=== PDF Generator ===")

file_list = os.listdir(save_DIR)

img_list = []
img_path = f'{save_DIR}/{file_list[0]}'
im_buf = Image.open(img_path)
cvt_rgb_0 = im_buf.convert('RGB')
for i in file_list:
    img_path = f'{save_DIR}/{i}'
    im_buf = Image.open(img_path)
    cvt_rgb = im_buf.convert('RGB')
    img_list.append(cvt_rgb)

del img_list[0]

savefilepath = f'./result/pdf/{save_NAME}'
try:
    if not os.path.exists(savefilepath):
        os.makedirs(savefilepath)
except OSError:
    print(f'Error: Cannot Find Select Directory: {savefilepath}')

cvt_rgb_0.save(f'{savefilepath}/{save_NAME}.pdf', save_all=True, append_images=img_list)
print("End of PDF Generator!!")

=== PDF Generator ===


IndexError: list index out of range

## 5. 사진 파일 삭제

In [ ]:
shutil.rmtree(save_DIR, ignore_errors=True)
print("사진 제거")
print("강의 요약 완료")